# フロントエンド層

フロントエンド（Front-end）層は、DNNにおいて入力データ（波形やスペクトログラムなど）を処理する最初の層です。

## 1D畳み込み層

その名の通り、1次元方向に畳み込みを行う層です。波形データをスペクトログラムに変換する代わりに、「学習可能（learnable）な」特徴抽出器と見なした畳み込み層に入力することで、よりタスクに適した特徴量を得る、という動機で使われます。

音楽音源分離モデルのフロントエンド層として多用されています[^LM19] [^demucs19]。

[^LM19]: Yi Luo and Nima Mesgarani. Conv-tasnet: surpassing ideal time-frequency magnitude masking for speech separation. IEEE ACM Trans. Audio Speech Lang. Process., 27(8):1256–1266, 2019. URL: https://doi.org/10.1109/TASLP.2019.2915167.

[^demucs19]: Alexandre Défossez, Nicolas Usunier, Léon Bottou, and Francis R. Bach. Demucs: deep extractor for music sources with extra unlabeled data remixed. CoRR, 2019. URL: http://arxiv.org/abs/1909.01174.

In [1]:
import torch
import torchaudio

wav, sr = torchaudio.load("assets/example1.ogg")
wav = wav.unsqueeze(0)
print('入力：', wav.shape)

conv_1d = torch.nn.Conv1d(
    in_channels=2,
    out_channels=64,
    kernel_size=5,
    stride=1,
    padding=2,  # stride=1, padding=(kernel_size - 1) // 2　にすれば、出力サイズが入力サイズと同じになる
    dilation=1,
    groups=1,
)
out = conv_1d(wav)
print('出力：', out.shape)



入力： torch.Size([1, 2, 3120768])
出力： torch.Size([1, 32, 3120768])


スペクトログラムの周波数次元を「チャンネル次元」と見なせば、スペクトログラムを入力とすることも可能です。

## Dilated-Conv-1D 

初めて「サンプル単位の逐次音声合成」という力技に挑戦したWaveNet[^wavenet16]にて提案された畳み込み層です。畳み込みカーネルを「飛ばし飛ばし」に適用することで、受容野（receptive field）をなるべく広げることが目的で、大域的な構造を考慮するようなモデルに使われます。

サンプル単位の音声合成は流石に効率が悪すぎましたが、Dilated-Conv-1D層を重ねたTCN（Temporal Convolutional Network）は音楽解析タスクで多用されています。

[^wavenet16]: Aäron van den Oord et al., Wavenet: A generative model for raw audio. In Alan W. Black, editor, The 9th ISCA Speech Synthesis Workshop, SSW 2016, Sunnyvale, CA, USA, September 13-15, 2016, 125. ISCA, 2016.

## SincNetのフロントエンド層